# Sequence Models

In this class, we relax the assumption that the data points are independently and identically distributed (i.i.d.) by moving to a scenario of structured prediction, where the inputs are assumed to have temporal or spacial dependencies. 

<b> Exercise 2.1 - Hidden Markov Models (HMM) </b>
<br>

Consider a person who is only interested in four activities: walking in the park (walk), shopping (shop), cleaning the apartment (clean) and playing tennis (tennis). Also, consider that the choice of what the person does on a given day is determined exclusively by the weather on that day, which can be either rainy or sunny. Now, supposing that we observe what the person did on a sequence of days, the question is: can we use that information to predict the weather on each of those days? To tackle this problem, we assume that the weather behaves as a discrete Markov chain: the weather on a given day depends only on the weather on the previous day. The entire system can be described as an HMM.
For example, assume we are asked to predict the weather conditions on two different sequences of days. During these two sequences, we observed the person performing the following activities:
<br>
<ul>
<li> “walk walk shop clean” </li>
<li> “clean walk tennis walk” </li>
</ul>
-> This will be our test set.
<br>
<br>
Moreover, and in order to train our model, we are given access to three different sequences of days, containing both
the activities performed by the person and the weather on those days, namely: 
<br>
<ul>
<li>“walk/rainy walk/sunny shop/sunny clean/sunny”</li>
<li>“walk/rainy walk/rainy shop/rainy clean/sunny”</li>
<li>“walk/sunny shop/sunny shop/sunny clean/sunny”</li>
</ul>
<br>
-> This will be our training set.

> Load the simple sequence dataset. From the ipython command line create a simple sequence object and look at the training and test set.

In [1]:
from __future__ import division
import lxmls.readers.simple_sequence as ssr
simple = ssr.SimpleSequence()

In [2]:
print "Train dataset:\n", simple.train
print "Test dataset:\n", simple.test

Train dataset:
[walk/rainy walk/sunny shop/sunny clean/sunny , walk/rainy walk/rainy shop/rainy clean/sunny , walk/sunny shop/sunny shop/sunny clean/sunny ]
Test dataset:
[walk/rainy walk/sunny shop/sunny clean/sunny , clean/sunny walk/sunny tennis/sunny walk/sunny ]


> Get in touch with the classes used to store the sequences, you will need this for the next exercise. Note that each label is internally stored as a number. This number can be used as index of an array to store information regarding that label.

In [3]:
for sequence in simple.train.seq_list: 
    print "Each sequence:", sequence, "\n"

print "\n"

for sequence in simple.train.seq_list: 
    print "Each sequence.x:", sequence.x, "\n"
    
print "\n"
    
for sequence in simple.train.seq_list: 
    print "Each sequence.y:", sequence.y, "\n"

Each sequence: walk/rainy walk/sunny shop/sunny clean/sunny  

Each sequence: walk/rainy walk/rainy shop/rainy clean/sunny  

Each sequence: walk/sunny shop/sunny shop/sunny clean/sunny  



Each sequence.x: [0, 0, 1, 2] 

Each sequence.x: [0, 0, 1, 2] 

Each sequence.x: [0, 1, 1, 2] 



Each sequence.y: [0, 1, 1, 1] 

Each sequence.y: [0, 0, 0, 1] 

Each sequence.y: [1, 1, 1, 1] 



So, the observactions correspond internally to: 
<br>[walk->0, shop->1, clean->2]
<br>[rainy->0, sunny->1]
    

<b> Exercise 2.2 - HMM Maximum Likelihood Training</b>
<br>

The provided function train supervised from the hmm.py file implements the above parameter estimates. Run this function given the simple dataset above and look at the estimated probabilities. 

In [4]:
import lxmls.sequences.hmm as hmmc
hmm = hmmc.HMM(simple.x_dict, simple.y_dict)
hmm.train_supervised(simple.train)

In [5]:
print "Initial Probabilities:", hmm.initial_probs, "\n"
print "Transition Probabilities:", hmm.transition_probs, "\n"
print "Final Probabilities:", hmm.final_probs, "\n"
print "Emission Probabilities", hmm.emission_probs, "\n"

Initial Probabilities: [0.66666667 0.33333333] 

Transition Probabilities: [[0.5   0.   ]
 [0.5   0.625]] 

Final Probabilities: [0.    0.375] 

Emission Probabilities [[0.75  0.25 ]
 [0.25  0.375]
 [0.    0.375]
 [0.    0.   ]] 



> Are they correct? You can also check the variables ending in  counts instead of  probs to see the raw counts (for example, typing hmm.initial counts will show you the raw counts of initial states). How are the counts related to the probabilities?

To answer the question, we can checks the implementation is correct with the following sanity checks:
<ul>
<li> Initial Counts: – Should sum to the number of sentences</li>
<li> Transition/Final Counts: – Should sum to the number of tokens </li>
<li> Emission Counts: – Should sum to the number of tokens </li>
</ul>

In [6]:
number_of_tokens=sum([len(seq) for seq in simple.train])

print "The inial counts are correct:",  sum(hmm.initial_counts)==len(simple.train.seq_list)
print "Transition and Final counts are correct:",hmm.transition_counts.sum()+ hmm.final_counts.sum()==number_of_tokens
print "Emission counts are correct:", hmm.emission_counts.sum()==number_of_tokens

The inial counts are correct: True
Transition and Final counts are correct: True
Emission counts are correct: True


<b> Exercise 2.3 - Decoding a Sequence </b>
<br> 
Convince yourself that the score of a path in the trellis (summing over the scores above) is equivalent to the log-probability log P(X = x, Y = y), as defined in Eq. 2.2. Use the given function compute scores on the first training sequence. Confirm that the values are correct. You should get the same values as presented below.

In [7]:
initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(simple.train.seq_list[0])
print "Initial scores", initial_scores, "\n"
print "Transition scores", transition_scores, "\n"
print "Final scores", final_scores, "\n"
print "Emission scores", emission_scores, "\n"

Initial scores [-0.40546511 -1.09861229] 

Transition scores [[[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]

 [[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]

 [[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]] 

Final scores [       -inf -0.98082925] 

Emission scores [[-0.28768207 -1.38629436]
 [-0.28768207 -1.38629436]
 [-1.38629436 -0.98082925]
 [       -inf -0.98082925]] 



lxmls/sequences/hmm.py:179: RuntimeWarning: divide by zero encountered in log
  transition_scores[pos-1, :, :] = np.log(self.transition_probs)
lxmls/sequences/hmm.py:177: RuntimeWarning: divide by zero encountered in log
  emission_scores[pos, :] = np.log(self.emission_probs[sequence.x[pos], :])
lxmls/sequences/hmm.py:182: RuntimeWarning: divide by zero encountered in log
  final_scores = np.log(self.final_probs)


<b> Exercise 2.4 - Computing in log-domain</b> 
<br>
Look at the module sequences/log domain.py. This module implements a function logsum pair(logx, logy) to add two numbers represented in the log-domain; it returns their sum also represented in the log-domain. The function logsum(logv) sums all components of an array represented in the log-domain. This will be used later in our decoding algorithms. To observe why this is important, type the following:

In [8]:
import numpy as np
a = np.random.rand(10) 

print np.log(sum(np.exp(a))) 
print np.log(sum(np.exp(10*a)))
print np.log(sum(np.exp(100*a)))
print np.log(sum(np.exp(1000*a)))

print "\nVS\n"
from lxmls.sequences.log_domain import *

print logsum(a)
print logsum(10*a)
print logsum(100*a)
print logsum(1000*a)

2.607942313490166
5.991010474846291
47.77049502782578
477.45974653821446

VS

2.6079423134901663
5.991010474846291
47.77049502782578
477.45974653821446


<b> Exercise 2.5 </b>
<br> Run the provided forward-backward algorithm on the first train sequence. Observe that both the forward and the backward passes give the same log-likelihood.

In [9]:
log_likelihood, forward = hmm.decoder.run_forward(initial_scores, transition_scores,
    final_scores, emission_scores)
print 'Log-Likelihood =', log_likelihood

Log-Likelihood = -5.068232326005127


In [10]:
log_likelihood, backward = hmm.decoder.run_backward(initial_scores, transition_scores,
    final_scores, emission_scores)
print 'Log-Likelihood =', log_likelihood

Log-Likelihood = -5.068232326005126


<b> Exercise 2.6 </b>
<br>
Compute the node posteriors for the first training sequence (use the provided compute posteriors func- tion), and look at the output. Note that the state posteriors are a proper probability distribution (the lines of the result sum to 1).

In [11]:
initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(simple.train.seq_list[0])
state_posteriors, _, _ = hmm.compute_posteriors(initial_scores,transition_scores,final_scores,emission_scores)
 
print state_posteriors

[[0.95738152 0.04261848]
 [0.75281282 0.24718718]
 [0.26184794 0.73815206]
 [0.         1.        ]]


<b> Exercise 2.7 </b>
<br>
Run the posterior decode on the first test sequence.

In [12]:
y_pred = hmm.posterior_decode(simple.test.seq_list[0])
print "Prediction test 0:", y_pred
print "Truth test 0:     ", simple.test.seq_list[0]

Prediction test 0: walk/rainy walk/rainy shop/sunny clean/sunny 
Truth test 0:      walk/rainy walk/sunny shop/sunny clean/sunny 


> Evaluate it.

In [13]:
def evaluate(y_seq, predicted_y_seq , number_of_classes):
    accuracy=0
    precision=0
    recall=0
    evaluations=0
    for class_i in number_of_classes:
        elements_correct= sum(1 for i in range(len(y_seq)) if predicted_y_seq[i]==class_i and predicted_y_seq[i]==y_seq[i])
        if elements_correct != 0:
            accuracy+=elements_correct #accuracy is the nº of elements correctly predicted of each class
            precision+=elements_correct/(predicted_y_seq == class_i).sum(0) #precision of each class
            recall+=elements_correct/ (y_seq).count(class_i)                #recall of class y
            evaluations+=1
    print "\nAccuracy:", accuracy, "\n"
    if evaluations !=0:
        print "Precision:", precision/evaluations, "\n"
        print "Recall:", recall/evaluations,"\n"

In [14]:
evaluate(simple.test.seq_list[0].y, y_pred.y, simple.y_dict.values())


Accuracy: 3 

Precision: 0.75 

Recall: 0.833333333333 



> Do the same for the second test sequence:

In [15]:
y_pred = hmm.posterior_decode(simple.test.seq_list[1])
print "Prediction test 1:", y_pred
print "Truth test 1:     ", simple.test.seq_list[1]

evaluate(simple.test.seq_list[1].y, y_pred.y, simple.y_dict.values())

Prediction test 1: clean/rainy walk/rainy tennis/rainy walk/rainy 
Truth test 1:      clean/sunny walk/sunny tennis/sunny walk/sunny 

Accuracy: 0 



lxmls/sequences/sequence_classifier.py:79: RuntimeWarning: invalid value encountered in subtract
  state_posteriors[pos, :] -= log_likelihood
lxmls/sequences/sequence_classifier.py:92: RuntimeWarning: invalid value encountered in double_scalars
  transition_posteriors[pos, state, prev_state] -= log_likelihood


> What is wrong? Note the observations for the second test sequence: the observation tennis was never seen at training time, so the probability for it will be zero (no matter what state). This will make all possible state sequences have zero probability. As seen in the previous lecture, this is a problem with generative models, which can be corrected using smoothing (among other options).

> <br> Change the train supervised method to add smoothing:

In [16]:
hmm.train_supervised(simple.train, smoothing=0.1) 

In [17]:
y_pred = hmm.posterior_decode(simple.test.seq_list[0])
print "Smoothing Prediction test 0:", y_pred
print "Smoothing Truth test 0:     ", simple.test.seq_list[0]
evaluate(simple.test.seq_list[0].y, y_pred.y, simple.y_dict.values())

Smoothing Prediction test 0: walk/rainy walk/rainy shop/sunny clean/sunny 
Smoothing Truth test 0:      walk/rainy walk/sunny shop/sunny clean/sunny 

Accuracy: 3 

Precision: 0.75 

Recall: 0.833333333333 



In [18]:
y_pred = hmm.posterior_decode(simple.test.seq_list[1])
print "Smoothing Prediction test 1:", y_pred
print "Smoothing Truth test 1:     ", simple.test.seq_list[1]

evaluate(simple.test.seq_list[1].y, y_pred.y, simple.y_dict.values())

Smoothing Prediction test 1: clean/sunny walk/sunny tennis/sunny walk/sunny 
Smoothing Truth test 1:      clean/sunny walk/sunny tennis/sunny walk/sunny 

Accuracy: 4 

Precision: 1.0 

Recall: 1.0 



<b> Exercise 2.8 - Viterbi </b>
<br>
Implement a method for performing Viterbi decoding in file sequence classification decoder.py.

In [19]:
#My implementation in the class decoder.py

def run_viterbi(self, initial_scores, transition_scores, final_scores, emission_scores):

    length = np.size(emission_scores, 0)  # Length of the sequence.
    num_states = np.size(initial_scores)  # Number of states.

    # Variables storing the Viterbi scores.
    viterbi_scores = np.zeros([length, num_states]) + logzero()

    # Variables storing the paths to backtrack.
    viterbi_paths = -np.ones([length, num_states], dtype=int)

    # Most likely sequence.
    best_path = -np.ones(length, dtype=int)

    # Initialization.
    viterbi_scores[0, :] = emission_scores[0, :] + initial_scores

    # Viterbi loop.
    for pos in xrange(1, length):
        for current_state in xrange(num_states):

            viterbi_scores[pos, current_state]= np.max(viterbi_scores[pos-1, :] + transition_scores[pos-1, current_state, :])
            viterbi_scores[pos, current_state]+=emission_scores[pos, current_state]
            viterbi_paths[pos, current_state] = np.argmax(viterbi_scores[pos-1, :] + transition_scores[pos-1, current_state, :])

    # Termination.
    best_score = np.max(viterbi_scores[length-1, :] + final_scores)

    #best state
    best_path[-1]=np.argmax(viterbi_scores[length-1, :] + final_scores) 

    for i in xrange(length-2,-1,-1):
        best_path[i]=viterbi_paths[i+1, best_path[i+1]]

    return best_path, best_score

In [21]:
hmm.train_supervised(simple.train, smoothing=0.1)
y_pred, score = hmm.viterbi_decode(simple.test.seq_list[0])
print "Viterbi decoding Prediction test 0 with smoothing:", y_pred, score, "\n"

print "Truth test 0:", simple.test.seq_list[0], "\n"

y_pred, score = hmm.viterbi_decode(simple.test.seq_list[1])
print "Viterbi decoding Prediction test 1 with smoothing:", y_pred, score, "\n"


Viterbi decoding Prediction test 0 with smoothing: walk/rainy walk/rainy shop/sunny clean/sunny  -6.020501246982869 

Truth test 0: walk/rainy walk/sunny shop/sunny clean/sunny  

Viterbi decoding Prediction test 1 with smoothing: clean/sunny walk/sunny tennis/sunny walk/sunny  -11.713974073970887 

